# Using [DSPy](https://github.com/stanfordnlp/dspy) 

- Signatures
- System Prompts


In this notebook we will show how to use DSPy Signatures and System prompts for gpt models.

# Import libraries

In [1]:
import os
import dspy

import openai

from dotenv import load_dotenv
# Load environment variables
load_dotenv()

openai.api_key = os.getenv('OPENAI_API_KEY')
if os.getenv('OPENAI_API_BASE'):
    openai.api_base = os.getenv('OPENAI_API_BASE')
if os.getenv('OPENAI_API_TYPE'):
    openai.api_type = os.getenv('OPENAI_API_TYPE')
if os.getenv('OPENAI_API_VERSION'):
    openai.api_version = os.getenv('OPENAI_API_VERSION')

In [2]:
# Set up the LM
turbo = dspy.OpenAI(model='gpt-3.5-turbo-instruct', max_tokens=250)
dspy.settings.configure(lm=turbo)

In [3]:
QUESTION = "What is the variational bicomplex?"

### Let's looks at a simple example of a Signature (similar to prompt template)

In [4]:
class ConciseQA(dspy.Signature):
    """You are a mathematics professor that answer's questions very concisely."""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="less than 30 words")

# Define the predictor.
generate_answer = dspy.Predict(ConciseQA)

# Call the predictor on a particular input.
pred = generate_answer(question=QUESTION)

# Print the input and the prediction.
print(f"Question: {QUESTION}")
print(f"Predicted Answer: {pred.answer}")

Question: What is the variational bicomplex?
Predicted Answer: A mathematical structure used to study variational problems, consisting of two complexes of differential forms.


In [5]:
turbo.history

[{'prompt': "You are a mathematics professor that answer's questions very concisely.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nAnswer: less than 30 words\n\n---\n\nQuestion: What is the variational bicomplex?\nAnswer:",
  'response': {'id': 'cmpl-95k52NVyVRCXlNMaFKcEFhZuENYvv',
   'choices': [{'finish_reason': 'stop',
     'index': 0,
     'logprobs': None,
     'text': ' A mathematical structure used to study variational problems, consisting of two complexes of differential forms.'}],
   'created': 1711155800,
   'model': 'gpt-3.5-turbo-instruct',
   'object': 'text_completion',
   'system_fingerprint': None,
   'usage': {'completion_tokens': 18,
    'prompt_tokens': 47,
    'total_tokens': 65}},
  'kwargs': {'temperature': 0.0,
   'max_tokens': 250,
   'top_p': 1,
   'frequency_penalty': 0,
   'presence_penalty': 0,
   'n': 1,
   'model': 'gpt-3.5-turbo-instruct',
   'prompt': "You are a mathematics professor that answer's questions very concisely.\n\n---\n\nFo

#### Now let's put this in a dspy Module class (works the same way)

In [6]:
class AnswerBot(dspy.Module):
    def __init__(self, tools=None):
        super().__init__()

        self.generate_answer = dspy.Predict(ConciseQA)
        
    def forward(self, question):
        prediction = self.generate_answer(question=f"{question}")
        return dspy.Prediction(answer=prediction.answer)

uncompiled_answer_bot = AnswerBot()
pred = uncompiled_answer_bot(question=QUESTION)
print(pred.answer)

A mathematical structure used to study variational problems, consisting of two complexes of differential forms.


In [7]:
turbo.history

[{'prompt': "You are a mathematics professor that answer's questions very concisely.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nAnswer: less than 30 words\n\n---\n\nQuestion: What is the variational bicomplex?\nAnswer:",
  'response': {'id': 'cmpl-95k52NVyVRCXlNMaFKcEFhZuENYvv',
   'choices': [{'finish_reason': 'stop',
     'index': 0,
     'logprobs': None,
     'text': ' A mathematical structure used to study variational problems, consisting of two complexes of differential forms.'}],
   'created': 1711155800,
   'model': 'gpt-3.5-turbo-instruct',
   'object': 'text_completion',
   'system_fingerprint': None,
   'usage': {'completion_tokens': 18,
    'prompt_tokens': 47,
    'total_tokens': 65}},
  'kwargs': {'temperature': 0.0,
   'max_tokens': 250,
   'top_p': 1,
   'frequency_penalty': 0,
   'presence_penalty': 0,
   'n': 1,
   'model': 'gpt-3.5-turbo-instruct',
   'prompt': "You are a mathematics professor that answer's questions very concisely.\n\n---\n\nFo

### Let's see if we can modify the OpenAI LM to use system prompts

**Note:** This code might not work until the [bug fix](https://github.com/stanfordnlp/dspy/pull/712) is in PyPi package.

In [4]:
from dsp.modules import LM
from dsp.modules.gpt3 import *

In [6]:
turbo4 = dspy.OpenAI(model='gpt-4-0125-preview', max_tokens=500, model_type='chat', system_prompt="You are a mathematics professor that answer's questions very concisely and starts all responses with: Hey hey!.")
dspy.settings.configure(lm=turbo4)

In [7]:
class ConciseQA(dspy.Signature):
    """You are a newly tenured mathematics professor."""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="less than 30 words")

# Define the predictor.
generate_answer = dspy.Predict(ConciseQA)

# Call the predictor on a particular input.
pred = generate_answer(question=QUESTION)

# Print the input and the prediction.
print(f"Question: {QUESTION}")
print(f"Predicted Answer: {pred.answer}")

Question: What is the variational bicomplex?
Predicted Answer: Hey hey! It's a mathematical structure used in the calculus of variations, organizing differential equations and variational problems on jet bundles systematically.


In [9]:
turbo4.history

[{'prompt': 'You are a newly tenured mathematics professor.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nAnswer: less than 30 words\n\n---\n\nQuestion: What is the variational bicomplex?\nAnswer:',
  'response': {'id': 'chatcmpl-9A2I3a3S0DbY4uv7lmxMTybZ8CzR1',
   'choices': [{'finish_reason': 'stop',
     'index': 0,
     'logprobs': None,
     'message': {'content': "Hey hey! It's a mathematical structure used in the calculus of variations, organizing differential equations and variational problems on jet bundles systematically.",
      'role': 'assistant',
      'function_call': None,
      'tool_calls': None}}],
   'created': 1712179111,
   'model': 'gpt-4-0125-preview',
   'object': 'chat.completion',
   'system_fingerprint': 'fp_13c70b9f70',
   'usage': {'completion_tokens': 27,
    'prompt_tokens': 75,
    'total_tokens': 102}},
  'kwargs': {'stringify_request': '{"temperature": 0.0, "max_tokens": 500, "top_p": 1, "frequency_penalty": 0, "presence_penalty": 0,